In [ ]:
import pandas as pd
import numpy as np

labels = pd.read_csv('labels.csv')

In [ ]:
# get dogbreed list
NUMBERCLASS = 120
selected_breed_list = labels.groupby('breed').count().sort_values('id',ascending=False).head(NUMBERCLASS).index
labels = labels[labels['breed'].isin(selected_breed_list)]

In [ ]:
#set breed_list index for labels
labels['index'] = 0
for i in range(NUMBERCLASS):
    labels.loc[labels['breed'] == selected_breed_list[i],'index'] = i

# vgg16 feature for softmax

In [ ]:
#get vgg16 feature for train 
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import sys,time
POOLING = 'avg'
data = []
model = VGG16(weights='imagenet', include_top=False,pooling=POOLING)
for i in labels.index:
    img_path = 'train/'+labels['id'][i]+'.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x,batch_size=32)
    data.append(features)
    sys.stdout.write('{0}/12222\r'.format(i + 1))
    sys.stdout.flush()
data_nd = np.asarray(data)


In [ ]:
# split data 
import keras
X = data_nd.reshape(data_nd.shape[0],-1)
y = labels['index']

split_pro = 0.8
split_edge = int(X.shape[0]*split_pro)
X_train = X[:split_edge]
y_train = y[:split_edge]
X_test = X[split_edge:]
y_test = y[split_edge:]


In [ ]:
#set model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=1)
logreg.fit(X_train,y_train)
logreg.score(X_test,y_test)

# Iception feature for softmax

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications import inception_v3
from keras.preprocessing import image
import keras
INPUT_SIZE = 299
data = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
y = keras.utils.to_categorical(labels['index'], num_classes=NUMBERCLASS)
for i,j in zip(labels.index,range(len(labels))):
    img_path = 'train/'+labels['id'][i]+'.jpg'
    img = image.load_img(img_path, target_size=(INPUT_SIZE, INPUT_SIZE))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = inception_v3.preprocess_input(x)
    data[j] = x

split_pro = 0.8
split_edge = int(data.shape[0]*split_pro)
X_train = data[:split_edge]
y_train = y[:split_edge]
X_test = data[split_edge:]
y_test = y[split_edge:]
print('Train Images shape: {} size: {:,}'.format(data.shape, data.size))


# Use Fine Tune

In [ ]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
POOLING = 'avg'
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False,pooling=POOLING)

# add a global spatial average pooling layer
x = base_model.output
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(NUMBERCLASS, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(X_train,y_train,batch_size=32,epochs=10)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(X_train,y_train,batch_size=32,epochs=10)

In [ ]:
score = model.evaluate(X_test,y_test)

In [ ]:
y_pred = model.predict(X_test,batch_size=128)

In [ ]:
np.sum(y_pred.argmax(axis=1) == (y_test*range(120)).sum(axis=1))*1.0/y_test.shape[0]

In [ ]:
import os
names = []
test_data = []
for f in os.listdir('test'):
    names.append(f[:-4])
    img_path = "test/"+f
    img = image.load_img(img_path, target_size=(INPUT_SIZE, INPUT_SIZE))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = inception_v3.preprocess_input(x)
    test_data.append(x)
test_data = np.asarray(test_data)
names = np.asarray(names)
test_data = test_data.reshape(test_data.shape[0],test_data.shape[2],test_data.shape[3],3)

In [ ]:
test_pred = model.predict(test_data)
submission = pd.DataFrame(data=test_pred,columns = np.asarray(selected_breed_list))
submission.insert(0,'id',names)
submission.to_csv('submission_version1.csv',index=False)